In [72]:
import numpy as np
import csv

x, y = [], []

with open("Results.csv") as r_file:
    file_reader = csv.reader(r_file, delimiter=',')
    for row in file_reader:
        split_row = row[0].split(';')
        if 'Пол' in split_row:
            continue
            
        temp_x = [split_row[i] for i in range(len(split_row)) if i in [1, 2, 10, 11, 12, 13, 14]]
        if temp_x[0] == 'Мужской':
            temp_x[0] = 1
        elif temp_x[0] == 'Женский':
            temp_x[0] = 0
        temp_x[1] = int(temp_x[1])
        
        for i in range(len(temp_x)):
            if temp_x[i] == 'Не играю':
                temp_x[i] = -1
            elif temp_x[i] == 'Иногда играю':
                temp_x[i] = 0
            elif temp_x[i] == 'Играю постоянно':
                temp_x[i] = 1
        x.append(temp_x)
        
        temp_y = split_row[3:10]
        for i in range(len(temp_y)):
            if temp_y[i] == 'Совсем не нравится' or temp_y[i] == 'Не нравится':
                temp_y[i] = 0
            elif temp_y[i] == 'Очень нравится' or temp_y[i] == 'Нравится':
                temp_y[i] = 1
        y.append(temp_y)

x, y = np.array(x), np.array(y)
x_train, x_test, y_train, y_test = x[:160], x[160:], y[:160], y[160:]

y_train_math, y_train_cs, y_train_phys, y_train_rus, y_train_liter, y_train_society, y_train_foreign = [[] for _ in range(7)]
y_test_math, y_test_cs, y_test_phys, y_test_rus, y_test_liter, y_test_society, y_test_foreign = [[] for _ in range(7)]

for i in y_train:
    y_train_math.append(i[0]); y_train_cs.append(i[1]); y_train_phys.append(i[2]); y_train_rus.append(i[3])
    y_train_liter.append(i[4]); y_train_society.append(i[5]); y_train_foreign.append(i[6])

for i in y_test:
    y_test_math.append(i[0]); y_test_cs.append(i[1]); y_test_phys.append(i[2]); y_test_rus.append(i[3])
    y_test_liter.append(i[4]); y_test_society.append(i[5]); y_test_foreign.append(i[6])

In [73]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        return self.value is not None

In [74]:
class DecisionTree:
    def __init__(self, max_depth=100, min_samples=10):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.tree = None
    
    def fit(self, x, y):
        self.tree = self.grow_tree(x, y)
    
    def predict(self, x):
        return np.array([self.travers_tree(i, self.tree) for i in x])
    
    def entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        entropy = -np.sum([p * np.log2(p) for p in ps if p > 0])
        gin = np.sum([p * (1 - p) for p in ps])
        return entropy
    
    def common(self, y):
        labels = np.unique(y)
        count = [np.count_nonzero(y == i) for i in labels]
        return labels[np.argmax(count)]
    
    def best_split(self, x, y):
        best_feature, best_threshold = None, None
        best_gain = -1
        
        for i in range(len(x[1])):
            thresholds = np.unique(x[:, i])
            for threshold in thresholds:
                gain = self.gain(x[:, i], y, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = i
                    best_threshold = threshold
        return best_feature, best_threshold
    
    def gain(self, x_column, y, threshold):
        if len(np.unique(y)) == 1: return 0
        
        n = len(y)
        parent = self.entropy(y)
        
        left_indexes = np.argwhere(x_column <= threshold).flatten() # Into the str
        right_indexes = np.argwhere(x_column > threshold).flatten()
        
        e_l, n_l = self.entropy(y[left_indexes]), len(left_indexes)
        e_r, n_r = self.entropy(y[right_indexes]), len(right_indexes)
        
        child = (n_l / n) * e_l + (n_r / n) * e_r
        return parent - child        

    def grow_tree(self, x, y, depth=0):
        n_samples = len(y)
        n_labels = len(np.unique(y))
        
        if n_labels == 1 or depth >= self.max_depth or n_samples <= self.min_samples:
            return Node(value=self.common(y))
        
        best_feature, best_threshold = self.best_split(x, y)
        
        left_indexes = np.argwhere(x[:, best_feature] <= best_threshold).flatten()
        right_indexes = np.argwhere(x[:, best_feature] > best_threshold).flatten()
        
        left = self.grow_tree(x[left_indexes, :], y[left_indexes], depth + 1)
        right = self.grow_tree(x[right_indexes, :], y[right_indexes], depth + 1)
        
        return Node(best_feature, best_threshold, left, right)
    
    def travers_tree(self, x, tree):
        if tree.is_leaf():
            return tree.value
        
        if x[tree.feature] < tree.threshold:
            return self.travers_tree(x, tree.left)
        return self.travers_tree(x, tree.right)

In [75]:
# Is gonna be used to predict the preferences for a single student (doesn't work correctly now)
class Student:
    def __init__(self, gender, age, shooters, puzzles, strategies, adventures, simulators):
        if gender == 'Мужской': gender = 1
        elif gender == 'Женский': gender = 0
        
        temp = [shooters, puzzles, strategies, adventures, simulators]
        if 'Не играю' in temp or 'Иногда играю' in temp or 'Играю постоянно' in temp:
            for i in range(len(temp)):
                if temp[i] == 'Не играю': temp[i] = -1
                elif temp[i] == 'Иногда играю': temp[i] = 0
                elif temp[i] == 'Играю постоянно': temp[i] = 1
        self.args = np.array([gender, age] + temp)
        
    def get_args(self):
        return self.args

In [76]:
# Learning the trees
clf_math, clf_cs, clf_phys, clf_rus, clf_liter, clf_society, clf_foreign = [DecisionTree() for _ in range(7)]
clf_math.fit(x_train, np.array(y_train_math))
clf_cs.fit(x_train, np.array(y_train_cs))
clf_phys.fit(x_train, np.array(y_train_phys))
clf_rus.fit(x_train, np.array(y_train_rus))
clf_liter.fit(x_train, np.array(y_train_liter))
clf_society.fit(x_train, np.array(y_train_society))
clf_foreign.fit(x_train, np.array(y_train_foreign))

In [77]:
# Testing
predict_math = clf_math.predict(x_test)
predict_cs = clf_cs.predict(x_test)
predict_phys = clf_phys.predict(x_test)
predict_rus = clf_rus.predict(x_test)
predict_liter = clf_liter.predict(x_test)
predict_society = clf_society.predict(x_test)
predict_foreign = clf_foreign.predict(x_test)

In [78]:
# Calculating the accuracy
math = np.sum(predict_math == y_test_math) / len(y_test_math)
cs = np.sum(predict_cs == y_test_cs) / len(y_test_cs)
phys = np.sum(predict_phys == y_test_phys) / len(y_test_phys)
rus = np.sum(predict_rus == y_test_rus) / len(y_test_rus)
liter = np.sum(predict_liter == y_test_liter) / len(y_test_liter)
society = np.sum(predict_society == y_test_society) / len(y_test_society)
foreign = np.sum(predict_foreign == y_test_foreign) / len(y_test_foreign)

print(math, cs, phys, rus, liter, society, foreign, sep='\n')

0.6944444444444444
0.6111111111111112
0.5
0.6666666666666666
0.7222222222222222
0.4444444444444444
0.5555555555555556
